In [ ]:
#Import required packages
import cv2 as cv
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torchvision.transforms as T

from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

#Insert filepath for local files
basepath = r"C:\Users\Thibaut Goldsborough\Documents\MLP2\data"
readpath = basepath + "\\Processed_black_background2"
filepath=r'C:\Users\Thibaut Goldsborough\Documents\MLP2\mlp-retinal\Cell_labels\labeled_cells_final.csv'


df = pd.read_csv(filepath,header=None)
df.columns = ['index', 'cell_id', 'label']
df1 = df[['cell_id', 'label']]
df1=df1.applymap(str)
df4=df1.iloc[1:]
#Check for duplicates
print(len(df4.cell_id.unique()))
print(len(df4))

data_dict={}
cells = []
num_images=37664
names=[]
for name in df4.cell_id:
    entry=name+'_Ch1.png'
    if os.path.isfile(os.path.join(readpath, entry)):
        data_dict[name]=[cv.imread(readpath+"\\"+name+'_Ch1.png',0  | cv.IMREAD_ANYDEPTH),
                cv.imread(readpath+"\\"+name+'_Ch6.png',0  | cv.IMREAD_ANYDEPTH),
                cv.imread(readpath+"\\"+name+'_Ch7.png',0  | cv.IMREAD_ANYDEPTH),
                cv.imread(readpath+"\\"+name+'_CNM.png',0  | cv.IMREAD_ANYDEPTH),
                cv.imread(readpath+"\\"+name+'_CCM.png',0  | cv.IMREAD_ANYDEPTH),
                #np.load(readpath+"\\"+name+ '_SCM.npy',allow_pickle=True)]
                cv.imread(readpath+"/"+name+'_SCM.png',0  | cv.IMREAD_ANYDEPTH)]
        
        names.append(name)
    if len(names)>=num_images:
         break


labels_to_onehot={}

labels_to_onehot["s"]=np.array([1.,0.,0.])
labels_to_onehot["d"]=np.array([0.,1.,0.])
labels_to_onehot["f"]=np.array([0.,0.,1.])

Channels=[5]  #Channel to be fed to the NN

# Slightly overkill method to make absolutely sure the images are in order
images_with_index = []
for image_i in data_dict:
    image=cv.merge([data_dict[image_i][i] for i in Channels])
    images_with_index.append((int(image_i),image))
    
images_with_index.sort()

images=[image[1] for image in images_with_index]
names=[image[0] for image in images_with_index]
labels=[labels_to_onehot[df4[df4.cell_id==str(name)].iloc[0].label] for name in names]  #This is so slowwwww, anyone got a better way of assigning the labels? around 1 min24.3s





In [ ]:

#Split data into training and test, just looking at first images now

train_test_split = 0.8

train_data1=images[:int(train_test_split*len(images))]
test_data1=images[int(train_test_split*len(images)):]

train_labels=labels[:int(train_test_split*len(images))]
test_labels=labels[int(train_test_split*len(images)):]

train_ID=names[:int(train_test_split*len(images))]
test_ID=names[int(train_test_split*len(images)):]

#Transform data into tensors, normalize images
transform_basic = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor()
])

# custom dataset
class CellDataset():
    def __init__(self, images,labels,ID, transforms=None):
        self.X = images
        self.Y=  labels
        self.Z= ID
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i]
        label=self.Y[i]
        ID=self.Z[i]
        data = np.asarray(data).astype(np.uint8)

        
        if self.transforms:
            data1 = self.transforms(data)
            data2 = self.transforms(data)
        
        return (data1,data2),np.array((np.argmax(label),ID))


train_data_basic = CellDataset(train_data1,train_labels,train_ID, transform_basic)
#Create DataLoaders
train_loader_basic = DataLoader(train_data_basic, batch_size=100, shuffle=False)

#data=next(iter(train_loader_basic))[0] Don't delete this is useful


def get_mean_std(loader):
    #https://stackoverflow.com/questions/48818619/pytorch-how-do-the-means-and-stds-get-calculated-in-the-transfer-learning-tutor
    mean = 0.
    std = 0.
    for images, _ in loader:
        images=images[0]
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    return mean, std


mean_loader,std_loader=get_mean_std(train_loader_basic)


#Transform data into tensors, normalize images
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=180,fill=37),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),
   # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
   transforms.Normalize(mean=[mean_loader], std=[std_loader])  # for grayscale images

])

#Transform data into tensors, normalize images
transform_test = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),
    #transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
   transforms.Normalize(mean=[mean_loader], std=[std_loader])  # for grayscale images
])


train_data = CellDataset(train_data1,train_labels,train_ID, transform_train)
test_data = CellDataset(test_data1,test_labels,test_ID, transform_test)

#Oversampling
from torch.utils.data.sampler import WeightedRandomSampler
counts=np.bincount(np.argmax(train_labels,axis=1))
labels_weights = 1. / counts
weights = labels_weights[np.argmax(train_labels,axis=1)]
sampler = WeightedRandomSampler(weights, len(weights))


#Create DataLoaders
batch_size = 100
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False,sampler=sampler)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=180,fill=37),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),

   transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
   ])
   
rotated = T.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),
    transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
    transforms.RandomRotation(degrees=180,fill=0),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(lambda x: x if 0.5>np.random.rand() else transforms.functional.invert(x)-1),
    transforms.Lambda(lambda x: x *(1+0.2*np.random.randn()) ),
    transforms.Lambda(lambda x: x + (0.1**0.5)*torch.randn(64, 64) )
    
    ]) 

train_data = CellDataset(train_data1,train_labels,train_ID, rotated)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)#,sampler=sampler)



In [ ]:
#RESNET18

from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor

#from .._internally_replaced_utils import load_state_dict_from_url
#from ..utils import _log_api_usage_once



def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 70,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        #_log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(len(Channels), self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)



def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any,
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    return _resnet("resnet18", BasicBlock, [2, 2, 2, 2], pretrained, progress, **kwargs)

In [ ]:
latent_dim=128
epochs=200

train_data = CellDataset(train_data1,train_labels,train_ID, rotated)
train_loader = DataLoader(train_data, batch_size=524,sampler=sampler)  #shuffle=True
test_loader = DataLoader(test_data, batch_size=524, shuffle=True)


import torch
from torch import nn
import torchvision
import copy

from lightly.data import LightlyDataset
from lightly.data import SimCLRCollateFunction
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLProjectionHead
from lightly.models.utils import deactivate_requires_grad
from lightly.models.utils import update_momentum


class BYOL(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = BYOLProjectionHead(latent_dim, 1024, 256)
        self.prediction_head = BYOLProjectionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z


resnet = resnet18(num_classes=latent_dim)
backbone = resnet #nn.Sequential(*list(resnet.children())[:-1])
model = BYOL(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

print("Starting Training")
for epoch in range(10):
    total_loss = 0
    for (x0, x1), _ in train_loader:
        update_momentum(model.backbone, model.backbone_momentum, m=0.99)
        update_momentum(model.projection_head, model.projection_head_momentum, m=0.99)
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(train_loader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
train_data = CellDataset(train_data1,train_labels,train_ID,rotated)
train_loader = DataLoader(train_data, batch_size=124, shuffle=True)#,sampler=sampler)

barlow=np.zeros((len(train_data1),latent_dim))
names=np.zeros((len(train_data1)))
labels=np.zeros((len(train_data1)))
i=0

for (x0, x1), Y in train_loader:
    x0 = x0.to(device)
    x1 = x1.to(device)
    latents=model.backbone(x0).detach().cpu().numpy()
    names[i:i+len(latents)]=Y[:,1]
    labels[i:i+len(latents)]=Y[:,0]
    barlow[i:i+len(latents)]=latents
    i+=len(latents)

    print(i,len(train_data1))

df3=pd.DataFrame(barlow)
df3["Cell_ID"]=names
df3.to_csv(basepath+"/Results/Barlow_latents"+str(train_losses[-1])+".csv", index = False, header=True)

    